# Forecsting remainders with OLS

## Libraries

In [22]:
install.packages("C:/Users/madsh/Downloads/path/to/withr_2.1.2.tar.gz", repos = NULL, type = "source")

library(tidyverse)
library(forecast)
library(ggplot2)
library(dplyr)
library(data.table)
library(IRdisplay)

library(foreach)
library(doParallel)

library(caret)
library(randomForest)
library(xgboost)


Warning message in install.packages("C:/Users/madsh/Downloads/path/to/withr_2.1.2.tar.gz", :
"installation of package 'C:/Users/madsh/Downloads/path/to/withr_2.1.2.tar.gz' had non-zero exit status"-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  


ERROR: Error: package or namespace load failed for 'tidyverse':
 .onAttach failed in attachNamespace() for 'tidyverse', details:
  call: NULL
  error: package or namespace load failed for 'ggplot2':
 .onAttach failed in attachNamespace() for 'ggplot2', details:
  call: NULL
  error: .onLoad failed in loadNamespace() for 'withr', details:
  call: unlockBinding("defer", <environment>)
  error: ingen binding for "defer"


## Custom functions

### Display tables

In [3]:
# Custom display function for the first and last 5 rows or full table if <= 20 rows
display_limited <- function(dt) {
  n <- nrow(dt)
  
  # If there are 20 or fewer rows, display the full table
  if (n <= 20) {
    limited_dt <- dt
  } else {
    # Otherwise, concatenate the first 5 rows, '...' and the last 5 rows
    limited_dt <- rbind(head(dt, 5), as.list(rep("...", ncol(dt))), tail(dt, 5))
  }
  
  # Generate raw HTML manually
  html_output <- paste0(
    "<table border='1' style='border-collapse:collapse;'>",
    "<thead><tr>",
    paste0("<th>", colnames(limited_dt), "</th>", collapse = ""),
    "</tr></thead>",
    "<tbody>",
    paste0(
      apply(limited_dt, 1, function(row) {
        paste0("<tr>", paste0("<td>", row, "</td>", collapse = ""), "</tr>")
      }),
      collapse = ""
    ),
    "</tbody></table>"
  )
  
  # Display the HTML in the Jupyter notebook
  display_html(html_output)
}


### Calculate metrics

In [4]:
calculate_metrics <- function(T_t, R_hat_t, individual) {
  # Ensure the inputs are numeric vectors and individual is a dataframe
  if (!is.numeric(T_t) || !is.numeric(R_hat_t)) {
    stop("Both T_t and R_hat_t need to be numeric vectors.")
  }
  
  # Calculate metrics
  mae <- mean(abs(T_t - R_hat_t), na.rm = TRUE)
  rmse <- sqrt(mean((T_t - R_hat_t)^2, na.rm = TRUE))
  mape <- mean(abs((T_t - R_hat_t) / T_t), na.rm = TRUE) * 100
  r_squared <- ifelse(all(T_t == R_hat_t), 1, summary(lm(T_t ~ R_hat_t))$r.squared)
  
  # Create a data frame to hold the metrics and values
  metrics_table <- data.frame(
    MAE = mae,
    RMSE = rmse,
    MAPE = mape,
    R_squared = r_squared
  )
  
  # Return the metrics table
  return(metrics_table)
}

### Data Preparation

In [5]:
prepare_X_t <- function(individual) {
  # Ensure the input is a dataframe
  if (!is.data.frame(individual)) {
    stop("The input must be a dataframe.")
  }
  
  # Extract hour from start_time and create a 'time_of_day' column
  individual$time_of_day <- format(as.POSIXct(individual$HourDK), "%H:%M:%S")
  
  # Exclude specified columns but keep 'time_of_day'
  X_t <- subset(individual, select = -c(HourDK, GrossConsumptionMWh))
  
  # Convert month, weekday, and time_of_day to factors with a reference category
  X_t$month <- relevel(as.factor(X_t$MonthOfYear), ref = "December")  # Set December as reference
  X_t$weekday <- relevel(as.factor(X_t$DayOfWeek), ref = "Sunday")   # Set Sunday as reference 
  X_t$time_of_day <- relevel(as.factor(X_t$Hour), ref = "0")         # Set 23 (11 PM) as reference

  # Remove original 'MonthOfYear', 'DayOfWeek', and 'Hour' columns to avoid duplication
  X_t <- subset(X_t, select = -c(MonthOfYear, DayOfWeek, Hour))
  
  # Create dummy variables for all factor columns (excluding reference levels)
  X_t <- model.matrix(~ . - 1, data = X_t)
  
  # Find the column indices for numerical columns AFTER creating dummy variables
  num_cols <- grep("^(Electric cars|Plug-in hybrid cars|humidity_past1h|temp_mean_past1h|wind_speed_past1h|EL_price)", colnames(X_t))
  
  # Standardize selected numerical columns
  X_t[, num_cols] <- apply(X_t[, num_cols], 2, 
                           function(x) (x - min(x)) / (max(x) - min(x)))
  
  # Return the processed dataframe
  return(as.data.frame(X_t))
}


### Lag and Align data by \\(h\\) (horizon)

In [6]:
lag_and_align_data <- function(X_t, R_t, h = 1) {
  # Validate inputs
  if (!is.numeric(R_t)) {
    stop("R_t should be a numeric vector.")
  }
  if (!is.data.frame(X_t) && !is.matrix(X_t)) {
    stop("X_t should be a dataframe or a matrix.")
  }
  if (!is.numeric(h) || h < 1) {
    stop("h should be a positive integer.")
  }
  
  # Convert X_t to a dataframe if it's a matrix
  if (is.matrix(X_t)) {
    X_t <- as.data.frame(X_t)
  }
  
  # Align R_t with the lagged X_t
  # Shift R_t by h positions to align with X_t from the previous timestep
  R_t_aligned <- R_t[(h + 1):length(R_t)]
  
  # Keep X_t up to the second to last row, so it aligns with the shifted R_t
  X_t_aligned <- X_t[1:(nrow(X_t) - h), ]
  
  # Return the aligned datasets
  list(X_t = X_t_aligned, R_t = R_t_aligned)
}

### Plot actual vs estimated

In [7]:
plot_actual_vs_estimated <- function(R_t, R_hat_t, individual) {
  # Validate input
  if (!is.numeric(R_t) || !is.numeric(R_hat_t)) {
    stop("R_t and R_hat_t should be numeric vectors.")
  }
  if (!is.data.frame(individual)) {
    stop("individual should be a dataframe.")
  }
    
  # Create the plot
  plot(R_t, type = 'l', col = 'blue', xlab = "Time", ylab = "Value", 
       main = "Actual vs. Estimated Time Series\nelvarme: %s, zip_code: %s")
  lines(R_hat_t, type = 'l', col = 'red')
  legend("topleft", legend = c("Actual", "Estimated"), col = c("blue", "red"), lty = 1)
}

## Loading data

In [8]:
##### Setting workign directory and loadign data #####
base_path <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data Cleaning"
setwd(base_path)
data <- fread(paste0(base_path,"/Output_file.csv"))
MSTL <- fread(paste0(base_path,"/MSTL_decomp_results.csv"))

ERROR: Error in fread(paste0(base_path, "/Output_file.csv")): could not find function "fread"


## Parameters

In [26]:
#train_size    <- 17544 #2 year training set
train_size    <- 8784  #1 year training set
num_timesteps <- 24
h             <- 1
total_size    <- nrow(data)-h
nrounds       <- 100
set.seed(42) 

In [13]:
anyNA(X_t[train_index, ])  # Tjek for manglende værdier
anyNA(T_t[train_index])    # Tjek for manglende værdier
anyNA(X_t[val_index, ])    # Tjek for manglende værdier
anyNA(T_t[val_index])      # Tjek for manglende værdier

[1] FALSE

[1] TRUE

[1] FALSE

[1] TRUE

### Data preparation

In [27]:
individual <- data
X_t <- prepare_X_t(as.data.frame(individual))
T_t <- as.matrix(MSTL$Trend, nrow = nrow(MSTL), ncol = 1)

lag_and_align <- lag_and_align_data(X_t, T_t, h = h)
X_t <- as.matrix(lag_and_align$X_t)
T_t <- as.numeric(lag_and_align$R_t)

## Hyper parameter tuning

In [28]:
# Simple train-validation split for hyperparameter tuning
train_index <- 1:train_size
val_index <- (train_size + 1):(train_size + num_timesteps)

dtrain <- xgb.DMatrix(data = X_t[train_index, ], label = T_t[train_index])
dval <- xgb.DMatrix(data = X_t[val_index, ], label = T_t[val_index])

watchlist <- list(train = dtrain, eval = dval)

tune_grid <- expand.grid(
  eta = c(0.01, 0.05, 0.1),
  max_depth = c(3, 6, 9),
  subsample = c(0.6, 0.8, 1.0),
  colsample_bytree = c(0.6, 0.8, 1.0)
)

best_params <- NULL
best_rmse <- Inf

for (i in 1:nrow(tune_grid)) {
  params <- list(
    objective = "reg:squarederror",
    eta = tune_grid$eta[i],
    max_depth = tune_grid$max_depth[i],
    subsample = tune_grid$subsample[i],
    colsample_bytree = tune_grid$colsample_bytree[i]
  )
  
  xgb_model <- xgb.train(
    params = params,
    data = dtrain,
    nrounds = nrounds,
    watchlist = watchlist,
    early_stopping_rounds = 5,
    verbose = 0
  )
  
  if (xgb_model$best_score < best_rmse) {
    best_rmse <- xgb_model$best_score
    best_params <- params
  }
}

### Fitting

In [30]:
no_cores <- detectCores() - 1
cl <- makeCluster(no_cores)
registerDoParallel(cl)

results <- foreach(j = seq(1, nrow(X_t) - train_size, by = num_timesteps), .combine = 'c', .packages = 'xgboost') %dopar% {
  start_index <- j
  end_index <- j + train_size - 1
  
  train_X_t <- X_t[start_index:end_index, ]
  train_T_t <- T_t[start_index:end_index]
  
  dtrain <- xgb.DMatrix(data = train_X_t, label = train_T_t)
  
  xgb_model <- xgb.train(params = best_params, data = dtrain, nrounds = 100)
  
  test_start_index <- end_index + 1
  test_end_index <- min(end_index + num_timesteps, total_size)
  test_X_t <- X_t[test_start_index:test_end_index, , drop = FALSE]
  dtest <- xgb.DMatrix(data = test_X_t)
  
  test_predictions <- predict(xgb_model, newdata = dtest)
  
  num_predictions_to_return <- min(num_timesteps, total_size - test_start_index + 1)
  return(test_predictions[1:num_predictions_to_return])
}
stopCluster(cl)

R_hat_t <- unlist(results)

### Calculating metrics

In [31]:
individual_metrics <- calculate_metrics(tail(T_t, n = length(T_t) - train_size), R_hat_t, data)
display_limited(individual_metrics)

MAE,RMSE,MAPE,R_squared
24.7536726364454,42.2786869106353,0.607440663659902,0.761563362954106


In [ ]:
# Define the file path
path_R <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/R_hat_t"
path_M <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Metrics"
# Automatically generate file names based on the parameters
file_name_R_hat_t <- file.path(path_R, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_XGB_R_hat_t.csv"))
file_name_metrics <- file.path(path_M, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_XGB_Metrics.csv"))



# Save R_hat_t as a CSV file
write.csv(R_hat_t, file = file_name_R_hat_t, row.names = FALSE)

# Save individual_metrics as a CSV file
write.csv(individual_metrics, file = file_name_metrics, row.names = FALSE)


In [ ]:
# Define the file path
path_R <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/R_hat_t"
path_M <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Metrics"
file_name_R_hat_t <- file.path(path_R, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_Boosting_R_hat_t.csv"))
file_name_metrics <- file.path(path_M, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_Boosting_Metrics.csv"))

